Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

***Задание 1.***

--Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

--Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [96]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
KEYWORDS = ['python', 'парсинг']

In [3]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')
# soup

In [9]:
posts = soup.find_all('article', class_='post_preview')
# len(posts)
# posts

In [7]:
d = []
for post in posts:
    post_id = post.parent.attrs.get('id')
   
    if not post_id:
        continue
    
    post_id = int(post_id.split('_')[-1])
    text_preview = post.find_all('div', class_='post__text')
    
    for preview in text_preview:
#         print(preview)
        preview_lower = preview.text.lower()
#         print(preview_lower) 
        if any([desired in preview_lower for desired in KEYWORDS]):
#             print(preview_lower)
                
            date_element = post.find('span', class_ = 'post__time')
            title_element = post.find('a', class_='post__title_link')
            link_element = title_element.attrs.get('href')
           
            d.append({'date': date_element.text, 'title': title_element.text, 'link': link_element})
            
df = pd.DataFrame(d)
df

,date,title,link
0,сегодня в 15:05,Как Django может обрабатывать 100 миллионов за...,https://habr.com/ru/post/512366/
1,сегодня в 13:01,"10 вещей, которые вы могли не знать о scikit-l...",https://habr.com/ru/company/plarium/blog/512332/


***Задание 2.***

--Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

--Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [222]:
import json
import datetime
EMAIL = ['example@gmail.com', 'example@mail.ru']
list_ = []

In [223]:
for i in range(0, len(EMAIL)):
    data_email = {'email': EMAIL[i]}
    data_email_json = json.dumps(data_email)
    req = requests.post('https://digibody.avast.com/v1/web/leaks', data = data_email_json)
    req_dict = req.json()
#     g = req_dict['value'][i]['description']
#     print(req_dict)
#     print(g)
    
    for i in range(0, len(req_dict['value'])):
    
        email = req_dict['value'][i]['username']
         
        leak_date = req_dict['value'][i]['leak_date']
        dt = datetime.datetime.fromtimestamp(leak_date / 1000)
        formatted_time = dt.isoformat(sep=' ', timespec='milliseconds')
        
        leak_source = req_dict['value'][i]['domain']
        
    #     leak_description = req_dict['value'][i]['description']
    
    #     list_.append({'почта': email, 'дата утечки': leak_date, 'источник утечки': leak_source, 'описание утечки': leak_description})
        list_.append({'почта': email, 'дата утечки': formatted_time, 'источник утечки': leak_source})

data = pd.DataFrame(list_)
data


,почта,дата утечки,источник утечки
0,example@gmail.com,2019-07-25 03:00:00.000,cafepress.com
1,example6,2020-07-23 03:00:00.000,liveauctioneers.com
2,camille_manring,2019-04-04 03:00:00.000,
3,example,2019-10-17 03:00:00.000,omgpop.com
4,fudgebob,2019-05-23 03:00:00.000,livejournal.com
...,...,...,...
201,example401,2019-03-07 03:00:00.000,dubsmash.com
202,example@mail.ru,2019-12-05 03:00:00.000,
203,laggerfe,2019-08-08 03:00:00.000,borealis.su
204,example@mail.ru,2019-02-06 03:00:00.000,
